# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


## Objectives


- Mark all launch sites on a map
- Mark the success/failed launches for each site on the map
- Calculate the distances between a launch site to its proximities

After completed the above tasks, we should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
# %pip install folium
# %pip install wget

In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, we can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and we should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


In [8]:
# Initial the map
site_map = folium.Map(location=[30.0, -100.0], zoom_start=5)
# For each launch site, add a Circle object and a Marker based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    circle = folium.Circle([row['Lat'], row['Long']], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site']))
    marker = folium.map.Marker(
        [row['Lat'], row['Long']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

# Show the map
site_map



Now, we can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?


# Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()

Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [12]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker([record['Lat'], record['Long']], icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

site_map

Zoom in and click in the launch stations to expand the markers as below:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, we can be able to easily identify which launch sites have relatively high success rates.


# Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway)


In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

We can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [16]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
launch_sites_df

# Create a new dataframe to add the closest markers coordinates
closest_markers_df = launch_sites_df.copy()

# Find the closest marker coordinates to the Launch Sites
# Closest coastlines coordinates:
coastline_lat = [28.5621, 28.56345, 28.60081, 34.6347]
coastline_lng = [-80.56772, -80.56797, -80.58723, -120.62535]

# Add closest coastlines coordinates to the closest_markers_df dataframe
for index, row in closest_markers_df.iterrows():
    closest_markers_df.loc[index, 'coastline_lat'] = coastline_lat[index]
    closest_markers_df.loc[index, 'coastline_lng'] = coastline_lng[index]

# Calculate the distance between the Launch Sites and the Coastlines
for index, row in closest_markers_df.iterrows():
    closest_markers_df.loc[index, 'distance_to_coastline_km'] = calculate_distance(row['Lat'], row['Long'], row['coastline_lat'], row['coastline_lng'])

closest_markers_df

,Launch Site,Lat,Long,coastline_lat,coastline_lng,distance_to_coastline_km
0,CCAFS LC-40,28.562302,-80.577356,28.56210,-80.56772,0.941685
1,CCAFS SLC-40,28.563197,-80.576820,28.56345,-80.56797,0.865034
2,KSC LC-39A,28.573255,-80.646895,28.60081,-80.58723,6.584372
3,VAFB SLC-4E,34.632834,-120.610745,34.63470,-120.62535,1.352621


After obtained its coordinate, create a `folium.Marker` to show the distance


In [17]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
for index, row in closest_markers_df.iterrows():
    distance_marker = folium.Marker(
        [row['coastline_lat'], row['coastline_lng']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(row['distance_to_coastline_km']),
            )
        )
    site_map.add_child(distance_marker)

Draw a `PolyLine` between a launch site to the selected coastline point


In [18]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
for index, row in closest_markers_df.iterrows():
    coordinates = [[row['Lat'], row['Long']], [row['coastline_lat'], row['coastline_lng']]]
    lines = folium.PolyLine(locations=coordinates, weight=1, tooltip='Distance to coastline')
    site_map.add_child(lines)


# Set a new map object to visualize the east coast launch sites and their distance to coastline
detail_map = folium.Map(location=[28.575, -80.605], zoom_start=14)
for layer in site_map._children.values():
    detail_map.add_child(layer)

detail_map

Similarly, we can draw a line betwee a launch site to its closest city, railway, highway, etc. We need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [19]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# Closest city coordinates:
city_lat = [28.40304, 28.40304, 28.60561, 34.6381]
city_lng = [-80.60079, -80.60079, -80.80713, -120.46977]

# Add closest city coordinates to the closest_markers_df dataframe
for index, row in closest_markers_df.iterrows():
    closest_markers_df.loc[index, 'city_lat'] = city_lat[index]
    closest_markers_df.loc[index, 'city_lng'] = city_lng[index]

# Calculate the distance between the Launch Sites and the closest city
for index, row in closest_markers_df.iterrows():
    closest_markers_df.loc[index, 'distance_to_city_km'] = calculate_distance(row['Lat'], row['Long'], row['city_lat'], row['city_lng'])

closest_markers_df

,Launch Site,Lat,Long,coastline_lat,coastline_lng,distance_to_coastline_km,city_lat,city_lng,distance_to_city_km
0,CCAFS LC-40,28.562302,-80.577356,28.56210,-80.56772,0.941685,28.40304,-80.60079,17.862215
1,CCAFS SLC-40,28.563197,-80.576820,28.56345,-80.56797,0.865034,28.40304,-80.60079,17.967735
2,KSC LC-39A,28.573255,-80.646895,28.60081,-80.58723,6.584372,28.60561,-80.80713,16.058235
3,VAFB SLC-4E,34.632834,-120.610745,34.63470,-120.62535,1.352621,34.63810,-120.46977,12.915112


In [20]:
# Create and add a folium.Marker on your selected closest city point on the map
# Display the distance between city point and launch site using the icon property
for index, row in closest_markers_df.iterrows():
    distance_marker = folium.Marker(
        [row['city_lat'], row['city_lng']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(row['distance_to_city_km']),
            )
        )
    site_map.add_child(distance_marker)


In [21]:
# Create a `folium.PolyLine` object using the city coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
for index, row in closest_markers_df.iterrows():
    coordinates = [[row['Lat'], row['Long']], [row['city_lat'], row['city_lng']]]
    lines = folium.PolyLine(locations=coordinates, weight=1, tooltip='Distance to city', color='red')
    site_map.add_child(lines)

# Set a new map object to visualize the east coast launch sites and their distance to city
detail_map = folium.Map(location=[28.5, -80.7], zoom_start=12)
for layer in site_map._children.values():
    detail_map.add_child(layer)

detail_map

After we plot distance lines to the proximities, we can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?


# Next Steps:

Now we have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, we will need to build a dashboard using Ploty Dash on detailed launch records.


Copyright © 2021 IBM Corporation. All rights reserved.
